- $d\geq 2$ is the input data dimension
- $k$ is the number of cluster per dimensions (over $2$ dimensions)
- $n$ is the number of training samples
- $m$ is the number of neurons
- $p$ is cluster labels
- sd number of spurious dimensions

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import torch
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
from scripts.architecture import MLP, MLPManual
from scripts.train_utils import AverageMeter, accuracy
from scripts.plot_utils import plot_loss_accuracy, plotValAccuracy, fillSubplot
from scripts.optimizer import Optimizer
from scripts.train import *
from scripts.data import * 

In [ ]:
learning_rate = 0.5
loss_type = "Binary Cross Entropy"
loss_fn = torch.nn.BCELoss()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
k = 3
num_epochs = 100
d = 3
sd = d - 3
sd = 0
n = 100
n_test = 4000

# Data

In [ ]:
# check why we have 3 - cluster always
trainset, testset = randomData(k, n, n_test, sd)

In [ ]:
X1 = trainset[:][0][trainset[:][1] == 1, :]
X2 = trainset[:][0][trainset[:][1] == 0, :]

plt.figure(figsize = (16,8))
plt.plot(X1[:,1],X1[:,2],"+r")
plt.plot(X2[:,1],X2[:,2],"_b")
plt.plot(cluster_center(torch.range(0,k**2),k)[0],cluster_center(torch.range(0,k**2),k)[1],"ok")
plt.axis("equal")
plt.axis("off")
plt.tight_layout()
plt.savefig("plots/randomData.png")
;

In [ ]:
sd = d - 3
n = 256
n_test = 4000
trainset, testset = randomData(k, n, n_test, sd)

In [ ]:
learning_rate = 0.5
batch_size = 1000
optim = "SGD"
measure_alignment = False
momentum, nesterov_momentum = False, False
weight_decay = None

In [ ]:
modelManual = MLPManual(d, learning_rate, loss_type, "BP", None, optim, device, measure_alignment, True, False)
trainLostList_sgd1_scratch, trainAccList_sgd1_scratch, \
valLossList_sgd1_scratch, valAccList_sgd1_scratch,_,_  = train_model_manually(modelManual, k, trainset, testset, loss_type, loss_fn, num_epochs, batch_size, momentum,
                                                                         nesterov_momentum, weight_decay, measure_alignment, n,d, validate_model = True, device=device, data="random")

plot_loss_accuracy(trainLostList_sgd1_scratch,valLossList_sgd1_scratch,trainAccList_sgd1_scratch,valAccList_sgd1_scratch,num_epochs)

In [ ]:
modelManual = MLPManual(d, learning_rate, loss_type, "BP", None, optim, device, measure_alignment, False, False)

trainLostList_sgd1_scratch, trainAccList_sgd1_scratch, \
valLossList_sgd1_scratch, valAccList_sgd1_scratch,_,_  = train_model_manually(modelManual, k, trainset, testset, loss_type, loss_fn, num_epochs, batch_size, momentum,
                                                                         nesterov_momentum, weight_decay, measure_alignment, n,d, validate_model = True, device=device, data="random")

plot_loss_accuracy(trainLostList_sgd1_scratch,valLossList_sgd1_scratch,trainAccList_sgd1_scratch,valAccList_sgd1_scratch,num_epochs)

In [ ]:
#Figure b
k=3
num_epochs = 250
d = 15
sd = d - 3
ns = [32,64,128,256,512]
n_test = 4000

learning_rate = 0.5
optim = "SGD"
measure_alignment = False
momentum, nesterov_momentum = False, False
weight_decay = None


In [ ]:
n = max(ns)
trainset, testset = randomData(k, n, n_test, 22)

In [ ]:
idx = 0
df = pd.DataFrame(columns=["Test Accuracy", "n", "Method"])

for n in ns:
    print("Number of data points:", n)
    modelManual1 = MLPManual(d, learning_rate, loss_type, "BP", None, optim, device, measure_alignment, True, False)
    modelManual2 = MLPManual(d, learning_rate, loss_type, "BP", None, optim, device, measure_alignment, False, False)
    modelManual3 = MLPManual(d, learning_rate, loss_type, "DFA", "uniform", optim, device, measure_alignment, True, False)
    results = {}
    for i in range(1,2):
        for model in [modelManual1, modelManual2, modelManual3]:        
            trainLostList_sgd1_scratch1, trainAccList_sgd1_scratch1, \
            valLossList_sgd1_scratch1, valAccList_sgd1_scratch1,_,_  = train_model_manually(model, k, trainset, testset, loss_type, loss_fn, num_epochs, n, momentum,
                                                                                    nesterov_momentum, weight_decay, measure_alignment, n,d, validate_model = True, device=device,
                                                                                    data="random")
            results[i] = valAccList_sgd1_scratch1

            liste = []
            for i in results:
                liste.append(results[i][-1])

            if model.train_method == "BP":
                if model.update_both == True:
                    method = "BP Both Layers"
                else:
                    method = "BP Output Layer"
            else:
                method = "DFA"

            for value in liste:
                df.loc[idx,:] = [value, n, method]
                idx += 1

df["Error"] = df["Test Accuracy"].apply(lambda x: 1-x)
df.to_csv("runs/randomData_n.csv", index=False)

In [ ]:
df_main = pd.read_csv("runs/randomData_n.csv")
ticks = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
tickLabels = map(str, ticks)
plt.figure(figsize=(12,6))

g = sns.pointplot(data=df_main, x="n", y="Error", hue="Layer", alpha=.6,)
g.legend_.set_title(None)

plt.ylabel("Test \nError", rotation=0, fontsize=12, labelpad=30)
plt.xlabel("n", rotation=0, fontsize=12, labelpad=30)
plt.grid()
plt.ylim(0,1)
plt.yticks([0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
plt.savefig("plots/randomData_n.png")
;

In [ ]:
n = 256
ds = [5,10,15,20,25]

In [ ]:
idx = 0
df = pd.DataFrame(columns=["Test Accuracy", "d", "Method"])

for d in ds:
    print("Number of dimensions points:", d)
    modelManual1 = MLPManual(d, learning_rate, loss_type, "BP", None, optim, device, measure_alignment, True, False)
    modelManual2 = MLPManual(d, learning_rate, loss_type, "BP", None, optim, device, measure_alignment, False, False)
    modelManual3 = MLPManual(d, learning_rate, loss_type, "DFA", "uniform", optim, device, measure_alignment, True, False)
    results = {}
    for i in range(1,2):
        for model in [modelManual1, modelManual2, modelManual3]:        
            trainLostList_sgd1_scratch1, trainAccList_sgd1_scratch1, \
            valLossList_sgd1_scratch1, valAccList_sgd1_scratch1,_,_  = train_model_manually(model, k, trainset, testset, loss_type, loss_fn, num_epochs, n, momentum,
                                                                                    nesterov_momentum, weight_decay, measure_alignment, n,d, validate_model = True, device=device,
                                                                                    data="random")
            results[i] = valAccList_sgd1_scratch1

            liste = []
            for i in results:
                liste.append(results[i][-1])

            if model.train_method == "BP":
                if model.update_both == True:
                    method = "BP Both Layers"
                else:
                    method = "BP Output Layer"
            else:
                method = "DFA"

            for value in liste:
                df.loc[idx,:] = [value, d, method]
                idx += 1

df["Error"] = df["Test Accuracy"].apply(lambda x: 1-x)
df.to_csv("runs/randomData_d.csv", index=False)

In [ ]:
df_main = pd.read_csv("runs/randomData_d.csv")
ticks = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
tickLabels = map(str, ticks)
plt.figure(figsize=(12,6))

g = sns.pointplot(data=df_main, x="d", y="Error", hue="Method", alpha=.6,)
g.legend_.set_title(None)

plt.ylabel("Test \nError", rotation=0, fontsize=12, labelpad=30)
plt.xlabel("n", rotation=0, fontsize=12, labelpad=30)
plt.grid()
plt.ylim(0,1)
plt.yticks([0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
plt.savefig("plots/randomData_d.png")
;

- Check why this problem (data) is important
- Try to replicate some results from the paper (it'll help to validate the correctnes of the implementation)
- Later move to comparasion between lazy method, BP and DFA (be careful about the)
    * Run 3-5 times and plot the final accuracy for different d
    * Plot them as in the paper
- After having the result of DFA if there's difference try to improve it with adaptive methods.
- Lastly refactor the code to make it suitable for random data experiments

- First replicate the result from the paper, plot b and c
- Late add the DFA